In [12]:
%pip install statsmodels


  Using cached statsmodels-0.14.5-cp313-cp313-macosx_11_0_arm64.whl.metadata (9.5 kB)
  Using cached patsy-1.0.1-py2.py3-none-any.whl.metadata (3.3 kB)
Using cached statsmodels-0.14.5-cp313-cp313-macosx_11_0_arm64.whl (9.7 MB)
Using cached patsy-1.0.1-py2.py3-none-any.whl (232 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 12.3 MB/s  0:00:01m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [statsmodels] [statsmodels]
Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install openpyxl


  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.


In [1]:
# 安装到当前内核；若已安装会自动跳过
try:
    import pandas, numpy, pytz  # noqa
except ModuleNotFoundError:
    %pip install -q pandas numpy pytz python-dateutil wrds


In [3]:
# ===== Jupyter-ready: WRDS + FRED + Ken French 合并，生成日频面板 & 0→+1 事件窗 =====
import os, io
from datetime import datetime
import pandas as pd
import numpy as np

# -------- 参数 --------
START = "2020-10-01"
END   = pd.Timestamp.today().strftime("%Y-%m-%d")
OUT_DAILY = "wrds_fomc_markets_daily.csv"     # 日频面板
OUT_EVENTS = "event_windows_0_1.csv"          # 若当前目录存在 events.csv 则生成

# -------- 尝试连接 WRDS（若失败则用公开源）--------
def connect_wrds():
    try:
        import wrds  # installed in Cell 1
        try:
            conn = wrds.Connection()  # 首次会提示账号密码
            print("[WRDS] connected.")
            return conn
        except Exception as e:
            print("[WRDS] connect failed, will use public fallbacks:", e)
            return None
    except Exception:
        print("[WRDS] package not present, will use public fallbacks.")
        return None

def wrds_has_library(conn, lib):
    try:
        return lib in conn.list_libraries()
    except Exception:
        return False

def try_wrds_get_fred_series(conn, sid):
    """从 WRDS.fred 拉 series；不同学校表结构不一，做了几种常见兜底。"""
    if conn is None or not wrds_has_library(conn, "fred"):
        return None
    sid_l = sid.lower()
    candidates = [sid_l, sid_l + "_daily", "series_" + sid_l]
    # 直接命中
    for t in candidates:
        try:
            df = conn.get_table(library="fred", table=t)
            dcol = next((c for c in df.columns if "date" in c.lower() or "observation" in c.lower()), df.columns[0])
            vcol = next((c for c in df.columns if sid_l in c.lower()), None)
            if vcol is None:
                num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
                if not num_cols: 
                    continue
                vcol = num_cols[-1]
            out = df[[dcol, vcol]].copy()
            out.columns = ["date", sid]
            out["date"] = pd.to_datetime(out["date"], errors="coerce")
            out[sid] = pd.to_numeric(out[sid], errors="coerce")
            out = out.dropna(subset=["date"]).sort_values("date")
            if len(out):
                print(f"[WRDS] fred.{t} -> {sid} ({len(out)} rows)")
                return out
        except Exception:
            pass
    # 扫描模式
    try:
        tabs = conn.raw_sql("select tablename from pg_catalog.pg_tables where schemaname='fred'")
        for t in tabs["tablename"].str.lower():
            if sid_l not in t: 
                continue
            try:
                df = conn.get_table(library="fred", table=t)
                dcol = next((c for c in df.columns if "date" in c.lower() or "observation" in c.lower()), df.columns[0])
                vcol = next((c for c in df.columns if sid_l in c.lower()), None)
                if vcol is None:
                    num_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
                    if not num_cols: 
                        continue
                    vcol = num_cols[-1]
                out = df[[dcol, vcol]].copy()
                out.columns = ["date", sid]
                out["date"] = pd.to_datetime(out["date"], errors="coerce")
                out[sid] = pd.to_numeric(out[sid], errors="coerce")
                out = out.dropna(subset=["date"]).sort_values("date")
                if len(out):
                    print(f"[WRDS] fred.{t} (scan) -> {sid} ({len(out)} rows)")
                    return out
            except Exception:
                pass
    except Exception:
        pass
    return None

def try_wrds_get_hml_daily(conn):
    """从 WRDS 的 ff/famafrench 拉日度 HML；成功返回 DataFrame(date,hml 小数)。"""
    if conn is None:
        return None
    for lib in ["ff", "famafrench", "ff_all"]:
        try:
            df = conn.get_table(library=lib, table="factors_daily")
            dcol = next((c for c in df.columns if "date" in c.lower()), df.columns[0])
            hcol = next((c for c in df.columns if c.lower()=="hml" or "hml" in c.lower()), None)
            if hcol is None: 
                continue
            out = df[[dcol, hcol]].copy()
            out.columns = ["date","hml"]
            out["date"] = pd.to_datetime(out["date"], errors="coerce")
            out["hml"]  = pd.to_numeric(out["hml"], errors="coerce")/100.0  # WRDS 通常是百分比
            out = out.dropna(subset=["date"]).sort_values("date")
            print(f"[WRDS] {lib}.factors_daily -> HML ({len(out)} rows)")
            return out
        except Exception:
            pass
    return None

# -------- 公开源（FRED/Ken French 官网）--------
def fred_csv(series_id, start=START, end=END):
    url = f"https://fred.stlouisfed.org/graph/fredgraph.csv?id={series_id}&cosd={start}&coed={end}"
    df = pd.read_csv(url)
    df.columns = ["date", series_id]
    df["date"] = pd.to_datetime(df["date"])
    df[series_id] = pd.to_numeric(df[series_id], errors="coerce")
    return df

def hml_from_web():
    url = "https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/ftp/F-F_Research_Data_Factors_daily_CSV.zip"
    df = pd.read_csv(url, compression="zip", skiprows=3)
    m = df.iloc[:,0].astype(str).str.contains("Copyright", na=False)
    if m.any(): 
        df = df.loc[~m].copy()
    df.columns = ["date","mktrf","smb","hml","rf"]
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d", errors="coerce")
    for c in ["mktrf","smb","hml","rf"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")/100.0
    return df[["date","hml"]]

# -------- 主流程 --------
conn = connect_wrds()

frames = {}
for sid in ["DGS10","DGS2","DGS1","DTWEXBGS"]:
    df = try_wrds_get_fred_series(conn, sid)
    if df is None:
        print(f"[INFO] {sid} -> using FRED public CSV")
        df = fred_csv(sid, START, END)
    df = df[(df["date"]>=pd.to_datetime(START)) & (df["date"]<=pd.to_datetime(END))]
    frames[sid] = df

fred_all = None
for sid, d in frames.items():
    fred_all = d if fred_all is None else fred_all.merge(d, on="date", how="outer")

hml = try_wrds_get_hml_daily(conn)
if hml is None:
    print("[INFO] HML -> using Ken French website ZIP")
    hml = hml_from_web()
hml = hml[(hml["date"]>=pd.to_datetime(START)) & (hml["date"]<=pd.to_datetime(END))]

daily = fred_all.merge(hml, on="date", how="outer").sort_values("date")
# 日历对齐 + 前向填充（节假日）
daily = daily.set_index("date").asfreq("D").ffill()

# 派生变量
daily["sprd_10y_2y"]   = daily["DGS10"] - daily["DGS2"]
daily["sprd_10y_2y_bp"]= daily["sprd_10y_2y"] * 100.0
daily["y1_bp"]         = daily["DGS1"] * 100.0
daily["usd_index"]     = daily["DTWEXBGS"]
daily["usd_ret_pct"]   = 100.0 * daily["usd_index"].pct_change()
daily["gv_ret_pct"]    = -100.0 * daily["hml"]   # Growth−Value = −HML（百分比）

daily_out = daily.reset_index()
cols = ["date","sprd_10y_2y_bp","y1_bp","usd_index","usd_ret_pct","gv_ret_pct",
        "DGS10","DGS2","DGS1","DTWEXBGS","hml"]
daily_out[cols].to_csv(OUT_DAILY, index=False)
print(f"[DONE] saved {OUT_DAILY} ({len(daily_out)} rows)")

display(daily_out.tail(5)[cols])



Enter your WRDS username [chichi]: tianyu03
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  y


Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
[WRDS] connected.
[INFO] DGS10 -> using FRED public CSV
[INFO] DGS2 -> using FRED public CSV
[INFO] DGS1 -> using FRED public CSV
[INFO] DTWEXBGS -> using FRED public CSV
[WRDS] ff.factors_daily -> HML (26066 rows)
[DONE] saved wrds_fomc_markets_daily.csv (1836 rows)


,date,sprd_10y_2y_bp,y1_bp,usd_index,usd_ret_pct,gv_ret_pct,DGS10,DGS2,DGS1,DTWEXBGS,hml
1831,2025-10-06,58.0,366.0,120.7028,0.154751,-0.0085,4.18,3.60,3.66,120.7028,0.000085
1832,2025-10-07,57.0,365.0,120.9097,0.171413,-0.0085,4.14,3.57,3.65,120.9097,0.000085
1833,2025-10-08,55.0,366.0,121.1884,0.230503,-0.0085,4.13,3.58,3.66,121.1884,0.000085
1834,2025-10-09,54.0,366.0,121.4965,0.254232,-0.0085,4.14,3.60,3.66,121.4965,0.000085
1835,2025-10-10,53.0,360.0,121.5217,0.020741,-0.0085,4.05,3.52,3.60,121.5217,0.000085


In [4]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# 1) 读取
factors = pd.read_csv("five_factor_scores_with_hawkindex.csv", parse_dates=["Date"])
markets = pd.read_csv("wrds_fomc_markets_daily.csv", parse_dates=["date"])

# 统一列名
factors = factors.rename(columns={"Date": "date"})
factors["date"] = pd.to_datetime(factors["date"]).dt.normalize()
markets["date"] = pd.to_datetime(markets["date"]).dt.normalize()

# 设为日频索引，缺口前向填充（确保能取到 d0/d1）
panel = (markets
         .sort_values("date")
         .set_index("date")
         .asfreq("D")
         .ffill())

# 2) 计算每个事件的 [0,+1] 变化
def next_available(idx, d):
    """取 d 当天；若无，则取之后第一个可用日；再兜底取最后一天"""
    if d in idx: return d
    pos = idx.searchsorted(d, side="left")
    return idx[min(pos, len(idx)-1)]

rows = []
for _, r in factors.sort_values("date").iterrows():
    d0 = pd.Timestamp(r["date"]).normalize()
    d0 = next_available(panel.index, d0)
    # d1 取 d0 之后的第一个可用日
    pos = panel.index.searchsorted(d0, side="right")
    d1 = panel.index[min(pos, len(panel.index)-1)]

    sprd0, sprd1 = panel.loc[d0, "sprd_10y_2y_bp"], panel.loc[d1, "sprd_10y_2y_bp"]
    y10_0, y10_1 = panel.loc[d0, "DGS10"], panel.loc[d1, "DGS10"]
    y2_0,  y2_1  = panel.loc[d0, "DGS2"],  panel.loc[d1, "DGS2"]
    y1_0,  y1_1  = panel.loc[d0, "y1_bp"], panel.loc[d1, "y1_bp"]
    usd0,  usd1  = panel.loc[d0, "usd_index"], panel.loc[d1, "usd_index"]
    gv0,   gv1   = panel.loc[d0, "gv_ret_pct"]/100.0, panel.loc[d1, "gv_ret_pct"]/100.0  # 百分比→小数

    rows.append({
        "date0": d0.date(),
        "date1": d1.date(),
        "d_sprd10y_2y_bp": float(sprd1 - sprd0),
        "d_y1_bp": float(y1_1 - y1_0),
        "d_usd_pct": float(100.0 * (usd1/usd0 - 1.0)),
        "gv_0to1_pct": float(100.0 * ((1+gv0)*(1+gv1) - 1.0)),
    } | r[["rate_score","inf_score","guidance_score","qt_score","growth_soft_score",
            "rate_score_z","inf_score_z","guidance_score_z","qt_score_z","growth_soft_score_z",
            "hawk_index","hawk_index_z","stance"]].to_dict())

ev = pd.DataFrame(rows)

# 3) 回归（默认用 z 分数；若想用原始分数，把 X_cols 改成无 _z 的版本）

X_cols = ["rate_score_z","inf_score_z","guidance_score_z","qt_score_z","growth_soft_score_z","hawk_index_z"]


def run_reg(df, y_col, x_cols=X_cols):
    data = df.dropna(subset=[y_col] + x_cols).copy()
    X = sm.add_constant(data[x_cols].astype(float))
    y = data[y_col].astype(float)
    mdl = sm.OLS(y, X).fit(cov_type="HC1")
    print(f"\n=== {y_col} ===")
    print(mdl.summary())
    return mdl

m_spread = run_reg(ev, "d_sprd10y_2y_bp")
m_y1     = run_reg(ev, "d_y1_bp")
m_usd    = run_reg(ev, "d_usd_pct")
m_gv     = run_reg(ev, "gv_0to1_pct")

# 4) 汇总成表
def pack(models, names):
    rows=[]
    for name, m in zip(names, models):
        for k in m.params.index:
            rows.append({
                "Y": name, "Var": k,
                "Coef": round(m.params[k],3),
                "SE": round(m.bse[k],3),
                "t": round(m.tvalues[k],2),
                "p": round(m.pvalues[k],3),
                "R2": round(m.rsquared,3)
            })
    return pd.DataFrame(rows)

summary = pack(
    [m_spread, m_y1, m_usd, m_gv],
    ["10Y–2Y Spread", "1Y Yield", "USD Index", "Growth–Value"]
)

summary.to_csv("five_factor_regression_summary.csv", index=False)
summary



=== d_sprd10y_2y_bp ===
                            OLS Regression Results                            
Dep. Variable:        d_sprd10y_2y_bp   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                 -0.026
Method:                 Least Squares   F-statistic:                     1.258
Date:                Thu, 16 Oct 2025   Prob (F-statistic):              0.303
Time:                        01:49:32   Log-Likelihood:                -123.34
No. Observations:                  40   AIC:                             260.7
Df Residuals:                      33   BIC:                             272.5
Df Model:                           6                                         
Covariance Type:                  HC1                                         
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const    

,Y,Var,Coef,SE,t,p,R2
0,10Y–2Y Spread,const,1.535,0.927,1.66,0.098,0.132
1,10Y–2Y Spread,rate_score_z,-523034.014,592401.147,-0.88,0.377,0.132
2,10Y–2Y Spread,inf_score_z,-523027.372,592401.966,-0.88,0.377,0.132
3,10Y–2Y Spread,guidance_score_z,-523034.223,592400.843,-0.88,0.377,0.132
4,10Y–2Y Spread,qt_score_z,-523030.195,592401.824,-0.88,0.377,0.132
5,10Y–2Y Spread,growth_soft_score_z,2092124.742,2369605.950,0.88,0.377,0.132
6,10Y–2Y Spread,hawk_index_z,1714439.962,1941826.024,0.88,0.377,0.132
7,1Y Yield,const,-1.907,0.669,-2.85,0.004,0.182
8,1Y Yield,rate_score_z,-321048.944,493987.351,-0.65,0.516,0.182
9,1Y Yield,inf_score_z,-321053.047,493988.666,-0.65,0.516,0.182
